In [14]:
import os
import pandas as pd

# Define the directory where the Excel files are stored
folder_path = r'Dataset\train'

In [15]:
dataframes = []


In [33]:

# Folder where your CSV files are stored
folder_path = 'Dataset/train'

# List of CSV files (only filenames, not full paths)
files = ['ex_1.csv', 'ex_20.csv', 'ex_21.csv', 'ex_24.csv', 'ex_9.csv', 'ex_23.csv']
dataframes = {}

# Loop through each file and read it into a dataframe
for file in files:
    file_path = os.path.join(folder_path, file)  # Create the full path to the file
    df = pd.read_csv(file_path, encoding='ascii')  # Use the full file path
    dataframes[file] = df  # Add dataframe to the dictionary
    
    # Convert 'time' column to numeric (if it's not already)
    df['time'] = pd.to_numeric(df['time'], errors='coerce')
    
    # Print information about each dataframe
    print(f"{file}:")
    print(f"Time range: {df['time'].min()} to {df['time'].max()}")
    print(f"Number of rows: {len(df)}")
    
    # Ensure there's no missing data in 'time' before calculating step
    if df['time'].isna().sum() == 0:
        print(f"Time step: {df['time'].diff().mode().values[0]:.2f}")
    else:
        print("Warning: There are missing values in the 'time' column. Time step calculation skipped.")
    
    print()

# Calculate total number of rows
total_rows = sum(len(df) for df in dataframes.values())
print(f"Total number of rows across all datasets: {total_rows}")
print(f"Target number of rows: 71225")
print(f"Missing rows: {71225 - total_rows}")


ex_1.csv:
Time range: 810.07028 to 10720.65858
Number of rows: 9920
Time step: 1.00

ex_20.csv:
Time range: 849.22 to 7824.0
Number of rows: 6495
Time step: 1.00

ex_21.csv:
Time range: 847.72 to 7776.6
Number of rows: 6495
Time step: 1.00

ex_24.csv:
Time range: 811.04 to 10579.6
Number of rows: 9023
Time step: 1.00

ex_9.csv:
Time range: 818.84 to 12636.84
Number of rows: 11819
Time step: 1.00

ex_23.csv:
Time range: 810.0 to 10954.52
Number of rows: 9188
Time step: 1.00

Total number of rows across all datasets: 52940
Target number of rows: 71225
Missing rows: 18285


In [34]:
# Find the overall time range
min_time = min(df['time'].min() for df in dataframes.values())
max_time = max(df['time'].max() for df in dataframes.values())

In [36]:
# Create a complete time series
import numpy as np

full_time_series = np.arange(min_time, max_time + 1, 1)


In [37]:

# Merge all dataframes
merged_df = pd.DataFrame({'time': full_time_series})
for file, df in dataframes.items():
    merged_df = pd.merge(merged_df, df, on='time', how='left', suffixes=('', f'_{file}'))


In [38]:
# Fill missing values with interpolation or forward fill
merged_df.interpolate(method='linear', inplace=True)
merged_df.fillna(method='ffill', inplace=True)

print(f"Shape of merged dataframe after filling missing values: {merged_df.shape}")
print("\
First few rows of the merged dataframe:")
print(merged_df.head().to_string())
print("\
Last few rows of the merged dataframe:")
print(merged_df.tail().to_string())

# Save the merged dataframe
merged_df.to_csv('merged_dataset_filled.csv', index=False)
print("\
Merged dataset with filled values saved as 'merged_dataset_filled.csv'")

Shape of merged dataframe after filling missing values: (11828, 13)
First few rows of the merged dataframe:
    time  input_voltage  el_power  input_voltage_ex_20.csv  el_power_ex_20.csv  input_voltage_ex_21.csv  el_power_ex_21.csv  input_voltage_ex_24.csv  el_power_ex_24.csv  input_voltage_ex_9.csv  el_power_ex_9.csv  input_voltage_ex_23.csv  el_power_ex_23.csv
0  810.0            NaN       NaN                      NaN                 NaN                      NaN                 NaN                      NaN                 NaN                     NaN                NaN                      3.0         1102.949693
1  811.0            NaN       NaN                      NaN                 NaN                      NaN                 NaN                      NaN                 NaN                     NaN                NaN                      3.0         1103.352691
2  812.0            NaN       NaN                      NaN                 NaN                      NaN                 N

C:\Users\Trisha\AppData\Local\Temp\ipykernel_17432\2201343337.py:3: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  merged_df.fillna(method='ffill', inplace=True)
